In [ ]:
!pip install scikit-optimize

In [ ]:
import numpy as np
import pandas as pd

from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from hyperopt import fmin, tpe, hp, anneal, Trials, space_eval

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
random_state = 42

In [ ]:
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = boston.target
train_data, test_data, train_targets, test_targets = train_test_split(X, y, test_size=0.20, shuffle=True, random_state=42)

In [ ]:
# Create tuples hidden_layer_sizes
hidden_layer_tuples = []
n_layers = [1, 2]
n_neurons = [10, 20, 30, 40, 50]
for layers in n_layers:
    for neurons in n_neurons:
        cur_size = (neurons,) * layers
        hidden_layer_tuples.append(cur_size)
print('The first hidden layer tuples:')
print(hidden_layer_tuples)

In [ ]:
all_val_scores = []
all_test_scores = []
n_iter = 100
num_folds=2
kf = KFold(n_splits=num_folds, shuffle=True, random_state=random_state)

In [ ]:
def calculate_min(scores):
    test_scores = []
    min_test_score = np.inf
    for score in scores:
        if score < min_test_score:
            min_test_score = score
        test_scores.append(min_test_score)
    return test_scores

In [ ]:
# minimal example
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical, Real
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.base import BaseEstimator, RegressorMixin
import itertools

# Ugly wrapper around MLPRegressor; The ugliness is to make parameters of the estimator be recognized properly
class MLPWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, layers=1, nodes=10, activation='logistic', solver='lbfgs', alpha='10**-5', batch_size=200, learning_rate='constant', learning_rate_init=10**-5):
        self.layers = layers
        self.nodes = nodes
        self.activation = activation
        self.solver = solver
        self.alpha = alpha
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.learning_rate_init = learning_rate_init

    def fit(self, X, y):
        hidden_layer = (self.nodes,)*self.layers
        model = MLPRegressor(
            hidden_layer_sizes=hidden_layer,
            solver=self.solver,
            alpha=self.alpha,
            batch_size=self.batch_size,
            learning_rate=self.learning_rate,
            learning_rate_init=self.learning_rate_init,
            random_state=42,
            verbose=False
        )

        model.fit(X, y)
        self.model = model
        return self

    def predict(self, X):
        return self.model.predict(X)

    def score(self, X, y):
        return cross_val_score(self.model, X, y, cv=kf, scoring="neg_mean_squared_error", n_jobs=-1).mean()

for random_state in range(5):
    reg = BayesSearchCV(
        estimator=MLPWrapper(),
        search_spaces={
            'layers': Categorical([1, 2]),
            'nodes': Categorical([10, 20, 30, 40, 50]),
            'activation': Categorical(['logistic', 'tanh', 'relu'], name='activation'),
            'solver': Categorical(['adam'], name='solver'),
            'alpha': Real(0.00001, 0.9999999, 'uniform', name='alpha'),
            'batch_size': Integer(2, 300, name='batch_size'),
            'learning_rate': Categorical(['constant', 'invscaling', 'adaptive'], name='learning_rate'),
            'learning_rate_init': Real(0.00001, 0.9999999, 'uniform', name='learning_rate_init')
        },
        n_iter=100,
        random_state=random_state
    )

    reg.fit(train_data, train_targets)

    reg_test_score = mean_squared_error(test_targets, reg.predict(test_data))
    all_test_scores.append(reg_test_score)

    print("Best MSE {:.3f} params {}".format(-reg.best_score_, reg.best_params_))

    all_val_scores.append(calculate_min(-reg.cv_results_['mean_test_score']))
    print(all_val_scores)


In [ ]:
df = pd.DataFrame(all_val_scores).transpose()
df.to_csv('full_results.csv', index=False)

In [ ]:
mean_std = []
for row_index in range(len(df)):
    row_np = df.iloc[row_index,:].to_numpy()
    mean_row = np.mean(row_np)
    std_row = np.std(row_np)
    mean_std.append((mean_row, std_row))
mean_std

In [ ]:
final_df = pd.DataFrame(mean_std, columns=['Mean', 'Std'])
final_df

In [ ]:
final_df.to_csv('GP - Final Results.csv', index=False)

In [ ]:
test_df = pd.DataFrame(all_test_scores, columns=['Test scores'])
test_df.to_csv('GP - Test scores.csv', index=False)